In [10]:
import requests
import re
import pandas as pd
from multiprocessing import Pool

import sys
sys.path.append('../')
from src.functions import get_prokaryotic_organisms, get_genes_for_organism, grouper, upload_s3_csv, import_s3_folder, fetch_info, check_list_length, check_if_prot_list

from random import randint
from time import sleep
import ast

from concurrent.futures import ThreadPoolExecutor
import threading
import csv 
import os
import glob
import tarfile


import s3fs

import multiprocessing as mp


In [12]:
"dEDZZDZDZ".lower()

'dedzzdzdz'

In [2]:
num_workers = mp.cpu_count()  
num_workers

128

In [4]:
s3_f_folder = 'KEGG_db/genomes'
local_path_f_new = "/home/onyxia/work/KEGG_Pipeline/newly_filled/"

#Create a directory for the new organisms
if not os.path.exists(local_path_f_new):
    os.makedirs(local_path_f_new)

In [8]:
#import_s3_csv(s3_f_folder, local_path_f_old)
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},key ='39CG0ODVMN8GP509UOP3', secret = 'WQdbeCAfnLkUv832HhJ5OMVmJ+o0jXFlDAXestpY', token = 'eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiIzOUNHME9EVk1OOEdQNTA5VU9QMyIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNzAwMTU2NDMwLCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6ImZsb3JpYW4uYmFzc29AZW5zYWUuZnIiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiZXhwIjoxNzAwMjQyODQxLCJmYW1pbHlfbmFtZSI6IkJhc3NvIiwiZ2l2ZW5fbmFtZSI6IkZsb3JpYW4iLCJncm91cHMiOltdLCJpYXQiOjE3MDAxNTY0NDAsImlzcyI6Imh0dHBzOi8vYXV0aC5sYWIuc3NwY2xvdWQuZnIvYXV0aC9yZWFsbXMvc3NwY2xvdWQiLCJqdGkiOiIxZTFkZDg0Yi04YjFkLTRmOWEtOTY0NS05MjZhZTI2YzNiMzUiLCJuYW1lIjoiRmxvcmlhbiBCYXNzbyIsInBvbGljeSI6InN0c29ubHkiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJnYW1lcjM1IiwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLXNzcGNsb3VkIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgcHJvZmlsZSBncm91cHMgZW1haWwiLCJzZXNzaW9uX3N0YXRlIjoiZGIwNTBhNmMtY2MzYS00NGUzLWJiNTYtNTk0ZWMwMGJmZGVlIiwic2lkIjoiZGIwNTBhNmMtY2MzYS00NGUzLWJiNTYtNTk0ZWMwMGJmZGVlIiwic3ViIjoiNWZjYmM0YTktOGY3Mi00OTMzLTkwMmYtM2JhY2UyMzI2OTYxIiwidHlwIjoiQmVhcmVyIn0.TXcI7nLMRMShAtLsm57JF98J5iGlwc4S5nC5YuZTaObZbJCAURKmN8V-a7Ux2sChUgasfx7E4pMpBBRwWQ7R0g')
file_list = fs.ls('gamer35/' + s3_f_folder)

In [9]:
prok = get_prokaryotic_organisms()
prok = [organism[1] for organism in  prok]

In [10]:
#On ne va chercher que les organismes non scrapés jusqu'ici 

scraped_genes =[]

for fname in file_list:
    gene = re.findall(r'gene_([a-z]+)', fname)
    scraped_genes.extend(gene)


print(len(scraped_genes))
print(f"Nous avons récupéré {100*len(scraped_genes)/len(prok)}% de la totalité des génomes de prokaryotistes disponibles sur KEGG")

8489
Nous avons récupéré 99.75323149236193% de la totalité des génomes de prokaryotistes disponibles sur KEGG


4898


Nous avons récupéré 58.15720731417716% de la totalité des génomes de prokaryotistes disponibles sur KEGG

In [11]:
# Find the intersection of the two lists


# Find the complementary of the intersection in list1
complementary = [item for item in prok if item not in scraped_genes]

print(len(complementary), len(prok) - len(scraped_genes))
prok = complementary

    

21 21


In [12]:
genes_orga = []
for organism in prok:
    sleep(randint(10,20))
    if requests.get(f'https://rest.kegg.jp/list/{organism}').text != '':
        genes_orga.append(organism)

In [13]:
prok = genes_orga

In [15]:
prok

['pvo', 'cwa', 'barc', 'barb', 'loki']

In [3]:
genomes = ['pant', 'lkf','rbn','psin','sene','smt','nfl','mvz']

In [4]:
def check_if_prot_list_API(page_response_text):

    pattern = re.compile(r'ORGANISM(.*?)NTSEQ', re.DOTALL)
    extracts = re.findall(pattern, page_response_text)
    indic_AA = [True if "AASEQ" in extract else False for extract in extracts]

    return indic_AA

def fetch_info_API(group):

    #regex
    pattern_KO = re.compile(r'ORTHOLOGY\s+(K\d{5})')
    pattern_seq = re.compile(r'NTSEQ\s+\d+(.*?)///', re.DOTALL)
    pattern_extract = re.compile(r'ENTRY(.*?)ORGANISM', re.DOTALL)

    #gene_ids
    gene_ids = [gene for gene in group if gene is not None]
    gene_group_url = '+'.join(gene_ids)
    gene_page_response = requests.get(f'https://rest.kegg.jp/get/{gene_group_url}')
    
    # Finding the KO
    match = re.findall(pattern_KO, gene_page_response.text)
    extracts = re.findall(pattern_extract, gene_page_response.text)
    indic_k0 = [True if "KO" in extract else False for extract in extracts]
    index_A = 0
    k0_value = []
    for element_B in indic_k0:
        if element_B and index_A < len(match):
            k0_value.append(match[index_A])
            index_A += 1
        else:
            k0_value.append('')
        
    #Finding the RNA NT sequence
    Nt_seq = re.findall(pattern_seq, gene_page_response.text)
    Nt_seq = [seq.replace(' ', '').replace('\n', '') for seq in Nt_seq]
    
    #Finding whether it is a protein or not
    prot = check_if_prot_list_API(gene_page_response.text)

    return gene_ids, k0_value, Nt_seq, prot


def scrape_genome_API(organism_code):
    genes = get_genes_for_organism(organism_code)
    organism = organism_code
    print(organism_code)
    dict_gene_org= []


    #we iterate over chunks of 10 genes_ids
    for i, group in enumerate(grouper(genes, 10)):
        
        results = {'gene_id': [], 'K0': [], 'Dna_SEQ': [], 'Prot': []}
        
        sleep(randint(20, 40))
        gene_ids, k0_value, Nt_seq, prot = fetch_info_API(group)
              
        results['gene_id'].extend(gene_ids)
        results['K0'].extend(k0_value)
        results['Dna_SEQ'].extend(Nt_seq)
        results['Prot'].extend(prot)

        dict_gene_org.append(results)
    df = pd.DataFrame(dict_gene_org)

    #Sometimes, due to high number of requests, requests get empty page, here we make sure that the dataframe is filled when written
    
    while ((df['K0'].apply(check_list_length)) | (df['Dna_SEQ'].apply(check_list_length)) | (df['Prot'].apply(check_list_length))).any():
        
        missing_info = df[df.apply(lambda row: any(check_list_length(cell) for cell in row[['K0', 'Dna_SEQ', 'Prot']]), axis=1)]
        
        for index, row in missing_info.iterrows():
            gene_ids_list = row['gene_id']

            # Fetch information for missing gene IDs
            sleep(randint(10,20))
            k0_values, dna_seq_values, prot_values = fetch_info_API(gene_ids_list)

            # Update the original DataFrame with the fetched information
            df.at[index, 'K0'] = k0_values
            df.at[index, 'Dna_SEQ'] = dna_seq_values
            df.at[index, 'Prot'] = prot_values

    df.to_csv(local_path_f_new + 'gene_' + organism +'_full.csv' , index= False, mode = 'w')
    #upload_s3_csv(s3_f_folder, local_path_f_new + 'gene_' + organism +'_full.csv' )
    return results

In [6]:
if __name__ == '__main__':
    prokaryotic_organisms2 = genomes # Your list of organisms

    # Create a Pool of workers for parallel processing
    with Pool(processes=5) as pool:  # You can adjust the number of processes as needed
        results = pool.map(scrape_genome_API, prokaryotic_organisms2)


lkf
rbn
pant
sene
psin


Process ForkPoolWorker-3:
Process ForkPoolWorker-5:


KeyboardInterrupt: 

In [ ]:
genes = scrape_organism_API('pvo')

pvo


In [88]:
response = requests.get(f'https://rest.kegg.jp/get/{gene_group_url}')

['pvo:PVOR_31966',
 'pvo:PVOR_r31966',
 'pvo:PVOR_29928',
 'pvo:PVOR_28674',
 'pvo:PVOR_28149',
 'pvo:PVOR_28134',
 'pvo:PVOR_25138',
 'pvo:PVOR_24971',
 'pvo:PVOR_20104',
 'pvo:PVOR_19184',
 'pvo:PVOR_19179',
 'pvo:PVOR_19174',
 'pvo:PVOR_18859',
 'pvo:PVOR_18844',
 'pvo:PVOR_15194',
 'pvo:PVOR_14539',
 'pvo:PVOR_09879',
 'pvo:PVOR_09834',
 'pvo:PVOR_05740',
 'pvo:PVOR_04203',
 'pvo:PVOR_r02179',
 'pvo:PVOR_23994',
 'pvo:PVOR_16704',
 'pvo:PVOR_19379',
 'pvo:PVOR_18849',
 'pvo:PVOR_28304',
 'pvo:PVOR_23674',
 'pvo:PVOR_13174',
 'pvo:PVOR_t11586',
 'pvo:PVOR_29474',
 'pvo:PVOR_18864',
 'pvo:PVOR_27674',
 'pvo:PVOR_26755',
 'pvo:PVOR_t03877',
 'pvo:PVOR_14834',
 'pvo:PVOR_30914',
 'pvo:PVOR_03893',
 'pvo:PVOR_t11588',
 'pvo:PVOR_26083',
 'pvo:PVOR_05378',
 'pvo:PVOR_t03879',
 'pvo:PVOR_19389',
 'pvo:PVOR_11600',
 'pvo:PVOR_00005',
 'pvo:PVOR_02181',
 'pvo:PVOR_18809',
 'pvo:PVOR_09884',
 'pvo:PVOR_t03881',
 'pvo:PVOR_29803',
 'pvo:PVOR_t11590',
 'pvo:PVOR_02521',
 'pvo:PVOR_09839',
 'pv

In [101]:
for i, group in enumerate(grouper(genes, 10)):
    
    results = {'gene_id': [], 'K0': [], 'Dna_SEQ': [], 'Prot': []}
    #gene_group = [gene for gene in group if gene is not None]
    gene_group_url = '+'.join(group)
    print(i, group)

0 ('pvo:PVOR_31966', 'pvo:PVOR_r31966', 'pvo:PVOR_29928', 'pvo:PVOR_28674', 'pvo:PVOR_28149', 'pvo:PVOR_28134', 'pvo:PVOR_25138', 'pvo:PVOR_24971', 'pvo:PVOR_20104', 'pvo:PVOR_19184')
1 ('pvo:PVOR_19179', 'pvo:PVOR_19174', 'pvo:PVOR_18859', 'pvo:PVOR_18844', 'pvo:PVOR_15194', 'pvo:PVOR_14539', 'pvo:PVOR_09879', 'pvo:PVOR_09834', 'pvo:PVOR_05740', 'pvo:PVOR_04203')
2 ('pvo:PVOR_r02179', 'pvo:PVOR_23994', 'pvo:PVOR_16704', 'pvo:PVOR_19379', 'pvo:PVOR_18849', 'pvo:PVOR_28304', 'pvo:PVOR_23674', 'pvo:PVOR_13174', 'pvo:PVOR_t11586', 'pvo:PVOR_29474')
3 ('pvo:PVOR_18864', 'pvo:PVOR_27674', 'pvo:PVOR_26755', 'pvo:PVOR_t03877', 'pvo:PVOR_14834', 'pvo:PVOR_30914', 'pvo:PVOR_03893', 'pvo:PVOR_t11588', 'pvo:PVOR_26083', 'pvo:PVOR_05378')
4 ('pvo:PVOR_t03879', 'pvo:PVOR_19389', 'pvo:PVOR_11600', 'pvo:PVOR_00005', 'pvo:PVOR_02181', 'pvo:PVOR_18809', 'pvo:PVOR_09884', 'pvo:PVOR_t03881', 'pvo:PVOR_29803', 'pvo:PVOR_t11590')
5 ('pvo:PVOR_02521', 'pvo:PVOR_09839', 'pvo:PVOR_28679', 'pvo:PVOR_03898', 'p

TypeError: sequence item 3: expected str instance, NoneType found

In [97]:
test = grouper(genes, 10)
print(test)

In [19]:
response = requests.get(f'https://rest.kegg.jp/get/pvo:PVOR_31966+pvo:PVOR_r31966+pvo:PVOR_29928+pvo:PVOR_28674+pvo:PVOR_28149+pvo:PVOR_28134+pvo:PVOR_25138+pvo:PVOR_24971+pvo:PVOR_20104+pvo:PVOR_19184')

In [20]:
response.text

'ENTRY       PVOR_31966        CDS       T05385\nNAME        (GenBank) stage II sporulation protein D\nORGANISM    pvo  Paenibacillus vortex\nPOSITION    complement(<1..486)\nDBLINKS     NCBI-ProteinID: EFU38046\n            UniProt: A0A2R9SKC3\nAASEQ       162\n            MKDSKVRVKVGIAGKERAMDPKTTPEEGATGQPYQVPAEGSAAPVLKWEFPGELEKTSKF\n            DKAHNHDSISDKKESEPGPRGHRGPARGPDAFGQSGHGPRAAHTPPPDRPAGAPAHGAAP\n            APGGPAAMPQGPHPAPEPQRYAQAAAGRAPRGGRTFRVRPAR\nNTSEQ       486\n            atgaaagattctaaggttcgggtgaaggtgggcattgcagggaaagaacgagcaatggat\n            ccgaagaccaccccggaggaaggggcaactgggcaaccctatcaagttccagcggaaggg\n            tcggcggctcccgtgttgaaatgggaatttcccggagaacttgagaagacatccaaattc\n            gacaaagcacataaccatgattcgatatcggacaaaaaagaatcggagcccgggccgcgc\n            gggcaccgcggcccggcgagagggccggacgccttcggccagtccggccatggcccgcgg\n            gcggcgcatacgccgccgcccgaccgccccgctggcgcccctgcgcatggcgccgcgcca\n            gcccccggcgggccagcggccatgccgcagggcccgcaccccgcgcctgagccgcagcgc

In [22]:
prot = check_if_prot_list_API(response.text)

In [23]:
prot

[True, False, True, True, True, True, True, True, True, True]

In [40]:
gene_group = genes[:10]

In [41]:
gene_group

['pvo:PVOR_31966',
 'pvo:PVOR_r31966',
 'pvo:PVOR_29928',
 'pvo:PVOR_28674',
 'pvo:PVOR_28149',
 'pvo:PVOR_28134',
 'pvo:PVOR_25138',
 'pvo:PVOR_24971',
 'pvo:PVOR_20104',
 'pvo:PVOR_19184']

In [30]:
gene_group_url = '+'.join(gene_group)
response = requests.get(f'https://rest.kegg.jp/get/{gene_group_url}')

In [73]:

pattern_seq = re.compile(r'NTSEQ\s+\d+(.*?)///', re.DOTALL)
seq = re.findall(pattern_seq, response.text)
seq = [seq.replace(' ', '').replace('\n', '') for seq in seq]
check_if_prot_list_API(response.text)

[True, False, True, True, True, True, True, True, True, True]

In [68]:
len(seq)

10

In [47]:
response.text

'ENTRY       PVOR_31966        CDS       T05385\nNAME        (GenBank) stage II sporulation protein D\nORGANISM    pvo  Paenibacillus vortex\nPOSITION    complement(<1..486)\nDBLINKS     NCBI-ProteinID: EFU38046\n            UniProt: A0A2R9SKC3\nAASEQ       162\n            MKDSKVRVKVGIAGKERAMDPKTTPEEGATGQPYQVPAEGSAAPVLKWEFPGELEKTSKF\n            DKAHNHDSISDKKESEPGPRGHRGPARGPDAFGQSGHGPRAAHTPPPDRPAGAPAHGAAP\n            APGGPAAMPQGPHPAPEPQRYAQAAAGRAPRGGRTFRVRPAR\nNTSEQ       486\n            atgaaagattctaaggttcgggtgaaggtgggcattgcagggaaagaacgagcaatggat\n            ccgaagaccaccccggaggaaggggcaactgggcaaccctatcaagttccagcggaaggg\n            tcggcggctcccgtgttgaaatgggaatttcccggagaacttgagaagacatccaaattc\n            gacaaagcacataaccatgattcgatatcggacaaaaaagaatcggagcccgggccgcgc\n            gggcaccgcggcccggcgagagggccggacgccttcggccagtccggccatggcccgcgg\n            gcggcgcatacgccgccgcccgaccgccccgctggcgcccctgcgcatggcgccgcgcca\n            gcccccggcgggccagcggccatgccgcagggcccgcaccccgcgcctgagccgcagcgc

In [42]:
lines = response.text.split('\n')
lines

['ENTRY       PVOR_31966        CDS       T05385',
 'NAME        (GenBank) stage II sporulation protein D',
 'ORGANISM    pvo  Paenibacillus vortex',
 'POSITION    complement(<1..486)',
 'DBLINKS     NCBI-ProteinID: EFU38046',
 '            UniProt: A0A2R9SKC3',
 'AASEQ       162',
 '            MKDSKVRVKVGIAGKERAMDPKTTPEEGATGQPYQVPAEGSAAPVLKWEFPGELEKTSKF',
 '            DKAHNHDSISDKKESEPGPRGHRGPARGPDAFGQSGHGPRAAHTPPPDRPAGAPAHGAAP',
 '            APGGPAAMPQGPHPAPEPQRYAQAAAGRAPRGGRTFRVRPAR',
 'NTSEQ       486',
 '            atgaaagattctaaggttcgggtgaaggtgggcattgcagggaaagaacgagcaatggat',
 '            ccgaagaccaccccggaggaaggggcaactgggcaaccctatcaagttccagcggaaggg',
 '            tcggcggctcccgtgttgaaatgggaatttcccggagaacttgagaagacatccaaattc',
 '            gacaaagcacataaccatgattcgatatcggacaaaaaagaatcggagcccgggccgcgc',
 '            gggcaccgcggcccggcgagagggccggacgccttcggccagtccggccatggcccgcgg',
 '            gcggcgcatacgccgccgcccgaccgccccgctggcgcccctgcgcatggcgccgcgcca',
 '            gcccccgg

In [34]:
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},key ='NBSJJCA4XW08TSHV7J93', secret = 'L4P0X+ahP1iyxFyQjWFzpyIPVUQkuJ1oDlbTRcMX', token = 'eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiJOQlNKSkNBNFhXMDhUU0hWN0o5MyIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNzAwNTcyNjkwLCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6ImZsb3JpYW4uYmFzc29AZW5zYWUuZnIiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiZXhwIjoxNzAwODQ5Nzc3LCJmYW1pbHlfbmFtZSI6IkJhc3NvIiwiZ2l2ZW5fbmFtZSI6IkZsb3JpYW4iLCJncm91cHMiOltdLCJpYXQiOjE3MDA3NjMzNzcsImlzcyI6Imh0dHBzOi8vYXV0aC5sYWIuc3NwY2xvdWQuZnIvYXV0aC9yZWFsbXMvc3NwY2xvdWQiLCJqdGkiOiJiZTJjMDQzNC1mMDY0LTQ0OTEtYjM3ZC0zMzgyYmFhMGI5OWIiLCJuYW1lIjoiRmxvcmlhbiBCYXNzbyIsInBvbGljeSI6InN0c29ubHkiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJnYW1lcjM1IiwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLXNzcGNsb3VkIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgcHJvZmlsZSBncm91cHMgZW1haWwiLCJzZXNzaW9uX3N0YXRlIjoiMzNjMzBjZDgtOWI1ZS00MTdjLTk3MjItMWM5ZjY1NDNhM2QwIiwic2lkIjoiMzNjMzBjZDgtOWI1ZS00MTdjLTk3MjItMWM5ZjY1NDNhM2QwIiwic3ViIjoiNWZjYmM0YTktOGY3Mi00OTMzLTkwMmYtM2JhY2UyMzI2OTYxIiwidHlwIjoiQmVhcmVyIn0.hs2XhlJOEfoP3CSn4EqpwLhKZ8vuuvDNpFqxBuOZ3n4l_bRbmFts_OsSUaupmP551td6rzOxDudja8pn3Sm51Q')

In [37]:
BUCKET='gamer35/'
s3_folder_path = s3_f_folder = 'KEGG_db/filled' +'/'

for file_path in glob.glob('/home/onyxia/work/KEGG_Pipeline/newly_filled/' + '*.csv'):        
        # Path to the local CSV file that you want to upload
    LOCAL_CSV_FILE = file_path  # Replace with the actual local CSV file path
    file_name_S3 = file_path.split('/')[-1]
        
    
        # Upload the CSV file to S3
    with open(LOCAL_CSV_FILE, 'rb') as local_file:
        with fs.open(BUCKET + s3_folder_path + file_name_S3, 'wb') as s3_file:
            s3_file.write(local_file.read())
        
    print(f"Uploaded {BUCKET + s3_folder_path + file_name_S3}")

Uploaded gamer35/KEGG_db/filled/gene_barb_full.csv
Uploaded gamer35/KEGG_db/filled/gene_loki_full.csv
Uploaded gamer35/KEGG_db/filled/gene_cwa_full.csv
Uploaded gamer35/KEGG_db/filled/gene_pvo_full.csv
Uploaded gamer35/KEGG_db/filled/gene_barc_full.csv


In [20]:
import polars as pl

In [27]:
df = pl.read_parquet("/home/onyxia/work/KEGG_Pipeline/newly_filled/alkg_genome.parquet")
df

shape: (416, 4)
┌────────────────────────┬────────────────────────┬────────────────────────┬───────────────────────┐
│ gene_id                ┆ K0                     ┆ Dna_SEQ                ┆ Prot                  │
│ ---                    ┆ ---                    ┆ ---                    ┆ ---                   │
│ list[str]              ┆ list[str]              ┆ list[str]              ┆ list[bool]            │
╞════════════════════════╪════════════════════════╪════════════════════════╪═══════════════════════╡
│ ["alkg:MOJ78_00005",   ┆ ["K02313", "K02338", … ┆ ["atggagaatattaacgattt ┆ [true, true, … false] │
│ "alkg:MOJ78…           ┆ ""]                    ┆ atggaataaa…            ┆                       │
│ ["alkg:MOJ78_00055",   ┆ ["K01985", "", …       ┆ ["tctggtgacgacggcgaaga ┆ [false, true, … true] │
│ "alkg:MOJ78…           ┆ "K01736"]              ┆ ggtcacaccc…            ┆                       │
│ ["alkg:MOJ78_00105",   ┆ ["K00817", "K04517", … ┆ ["atgaaggtgaaaaaacagtt ┆ [true, true, … true]  │
│ "alkg:MOJ78…           ┆ ""]                    ┆ gttgtcacta…            ┆                       │
│ ["alkg:MOJ78_00155",   ┆ ["K01990", "K01992", … ┆ ["atgagtgatacatatgcaat ┆ [true, true, … true]  │
│ "alkg:MOJ78…           ┆ "K02033"]              ┆ cgagttgaaa…            ┆                       │
│ …                      ┆ …                      ┆ …                      ┆ …                     │
│ ["alkg:MOJ78_20605",   ┆ ["K04096", "K00783", … ┆ ["ttgaatataaaaaatactct ┆ [true, true, … true]  │
│ "alkg:MOJ78…           ┆ ""]                    ┆ gttgctttta…            ┆                       │
│ ["alkg:MOJ78_20655",   ┆ ["", "", … "K03111"]   ┆ ["ggctcgatagctcagtcggt ┆ [false, false, …      │
│ "alkg:MOJ78…           ┆                        ┆ agagcagagg…            ┆ true]                 │
│ ["alkg:MOJ78_20705",   ┆ ["K02990", "K06942", … ┆ ["atgcgtagatatgaaatcat ┆ [true, true, … true]  │
│ "alkg:MOJ78…           ┆ "K03496"]              ┆ gtacattctg…            ┆                       │
│ ["alkg:MOJ78_20755",   ┆ ["K03497", "K03501", … ┆ ["atgaaacaaccgttttcacg ┆ [true, true, … true]  │
│ "alkg:MOJ78…           ┆ "K02914"]              ┆ tttgtttgga…            ┆                       │
└────────────────────────┴────────────────────────┴────────────────────────┴───────────────────────┘

In [44]:
import os
len(os.listdir("/home/onyxia/work/KEGG_Pipeline/newly_filled/"))

136

## Further tests and experiments

In [ ]:
"""

#We convert our folder Proka_genes in .tar.gz file before

!tar -czf archive.tar.gz /home/onyxia/work/Proka_genes

#Exporting and storing the .tar.gz file containing the to_be_filled_genes to the appropriate folder in our S3 bucket

BUCKET_OUT = "gamer35/"
FILE_KEY_OUT_S3 = "KEGG_database/to_be_filled/archive.tar.gz"
FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3
LOCAL_TAR_GZ_FILE = '/home/onyxia/work/Kegg_Explo/archive.tar.gz'


with open(LOCAL_TAR_GZ_FILE, 'rb') as local_file:
    with fs.open(FILE_PATH_OUT_S3, 'wb') as s3_file:
        s3_file.write(local_file.read())

print(f"Uploaded {LOCAL_TAR_GZ_FILE} to S3://{BUCKET_OUT}/{FILE_KEY_OUT_S3}")

"""

In [ ]:
"""
##### Ce code permet d'uploader en boucle des fichiers csv dans un bucket S3 ####

# Define the path for the CSV file on your S3 bucket
S3_BUCKET_NAME = 'gamer35/'
S3_FILE_PATH = 'KEGG_database/waiting_to_be_added2/'  # Replace with your desired path and file name
local_path = "/home/onyxia/work/KEGG_pipeline/Proka_genes/"

S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

for fname in glob.glob(local_path + '*.csv'):
    
    # Path to the local CSV file that you want to upload
    LOCAL_CSV_FILE = fname  # Replace with the actual local CSV file path
    file_name_S3 = fname.split('/')[-1]
    

    # Upload the CSV file to S3
    with open(LOCAL_CSV_FILE, 'rb') as local_file:
        with fs.open(S3_BUCKET_NAME + S3_FILE_PATH+ file_name_S3, 'wb') as s3_file:
            s3_file.write(local_file.read())
    
    print(f"Uploaded {S3_BUCKET_NAME + S3_FILE_PATH+ file_name_S3}")
"""

In [ ]:
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "gamer35/KEGG_database/waiting_to_be_added2"
fs.ls(BUCKET)


In [ ]:
for fname in glob.glob(path + '*.csv'):
    gene = re.findall(r'gene_([a-z]+)\.csv', fname)
    print(gene[0])

In [ ]:
genes = get_genes_for_organism("eco")
print(genes)

In [ ]:
df_exploded_gene =  df_gene.explode(['gene_id', 'K0', 'Dna_SEQ'])

In [ ]:
"""
Ici nous avons des différences dues au fait que nombre de gene % 10 diffère selon les espèces
Cependant problème : il y a des différences entre colonnes, pourquoi?
"""
df_tests = df_gene

df_tests['gene_id_count'] = df_tests['gene_id'].apply(len)
df_tests['K0_count'] = df_tests['K0'].apply(len)
df_tests['Dna_SEQ_count'] = df_tests['Dna_SEQ'].apply(len)

# Now, you can examine the counts for each column
print("Element counts in 'gene_id':", df_tests['gene_id_count'].unique())
print("Element counts in 'K0':", df_tests['K0_count'].unique())
print("Element counts in 'Dna_SEQ':", df_tests['Dna_SEQ_count'].unique())


In [ ]:
sum(df_tests['Dna_SEQ_count']==0)

In [ ]:
sum(df_tests['K0_count']==0)

In [ ]:
df_tests.loc[df_tests['Dna_SEQ_count'] == 0]

In [ ]:
import pandas as pd

# Assuming you have a DataFrame df_gene
# You want to explode the 'gene_id', 'K0', and 'Dna_SEQ' columns

# Create a temporary DataFrame for each column and explode it
df_gene_id = df_gene[['gene_id']].explode('gene_id').reset_index(drop=True)
df_k0 = df_gene[['K0']].explode('K0').reset_index(drop=True)
df_dna_seq = df_gene[['Dna_SEQ']].explode('Dna_SEQ').reset_index(drop=True)

# Concatenate the resulting DataFrames
result_df = pd.concat([df_gene_id, df_k0, df_dna_seq], axis=1)



In [ ]:
df_gene

In [ ]:
result_df